In [25]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd



In [26]:
df = pd.read_csv('HousingPrices.csv')
X = df.drop(columns=['SalePrice'])
ydf = df[['SalePrice']]
df.head()



,YearBuilt,2ndFlrSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,YrSold,SalePrice
0,2003,854,1710,2,1,3,8,2008,208500
1,1976,0,1262,2,0,3,6,2007,181500
2,2001,866,1786,2,1,3,6,2008,223500
3,1915,756,1717,1,0,3,7,2006,140000
4,2000,1053,2198,2,1,4,9,2008,250000


In [27]:
print(len(X),len(ydf))

1460 1460


In [28]:

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, ydf, test_size = 0.2)

In [29]:
xval, x_test, yval, y_test = train_test_split(xtest, ytest, test_size = 0.5)


     

In [30]:

    
    model = keras.Sequential()
    model.add(keras.layers.Dense(units= 8,activation= 'relu', input_shape = (8,)))
    model.add(keras.layers.Dense(units= 8, activation= 'relu'))
    model.add(keras.layers.Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc'])
    model.summary()
   

Model: &quot;sequential_2&quot;
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 9         
Total params: 153
Trainable params: 153
Non-trainable params: 0
_________________________________________________________________


In [31]:
 model.fit(xtrain,ytrain, epochs=2000,validation_data=(xval,yval) )
    

795200.0000 - val_acc: 0.0000e+00
Epoch 1877/2000
37/37 [==============================] - 0s 1ms/step - loss: 2690411776.0000 - acc: 0.0000e+00 - val_loss: 1995948288.0000 - val_acc: 0.0000e+00
Epoch 1878/2000
37/37 [==============================] - 0s 1ms/step - loss: 2689837312.0000 - acc: 0.0000e+00 - val_loss: 1995671936.0000 - val_acc: 0.0000e+00
Epoch 1879/2000
37/37 [==============================] - 0s 1ms/step - loss: 2691956480.0000 - acc: 0.0000e+00 - val_loss: 1997536384.0000 - val_acc: 0.0000e+00
Epoch 1880/2000
37/37 [==============================] - 0s 1ms/step - loss: 2693609984.0000 - acc: 0.0000e+00 - val_loss: 1995379328.0000 - val_acc: 0.0000e+00
Epoch 1881/2000
37/37 [==============================] - 0s 1ms/step - loss: 2691308032.0000 - acc: 0.0000e+00 - val_loss: 1996138368.0000 - val_acc: 0.0000e+00
Epoch 1882/2000
37/37 [==============================] - 0s 1ms/step - loss: 2690451200.0000 - acc: 0.0000e+00 - val_loss: 1994839040.0000 - val_acc: 0.0000e+00


&lt;tensorflow.python.keras.callbacks.History at 0x20c4ef779c8&gt;

In [80]:
xval.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,sqft_above,sqft_basement,street,city
860,2.0,1.0,1490,3825,1.0,3,860,630,7008 19th Ave NW,Seattle
3836,5.0,1.0,1590,6700,1.5,3,1090,500,10626 12th Ave SW,Seattle
1891,3.0,2.5,1520,2208,2.0,3,1040,480,2537 13th Ave W,Seattle
4572,6.0,4.5,3830,4800,3.0,3,3050,780,2425 3rd Ave W,Seattle
4453,3.0,1.0,1300,6710,1.0,4,1300,0,2760 72nd Ave SE,Mercer Island


In [93]:
df.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,sqft_above,sqft_basement,street,city
0,3.0,1.50,1340,7912,1.5,3,1340,0,18810 Densmore Ave N,Shoreline
1,5.0,2.50,3650,9050,2.0,5,3370,280,709 W Blaine St,Seattle
2,3.0,2.00,1930,11947,1.0,4,1930,0,26206-26214 143rd Ave SE,Kent
3,3.0,2.25,2000,8030,1.0,4,1000,1000,857 170th Pl NE,Bellevue
4,4.0,2.50,1940,10500,1.0,4,1140,800,9105 170th Ave NE,Redmond


In [18]:
x_test.head()


,YearBuilt,2ndFlrSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,YrSold
1053,1957,0,1526,1,0,4,7,2010
864,2007,0,1372,2,0,3,6,2008
185,1892,1518,3608,2,1,4,12,2006
631,2006,0,1554,2,0,2,6,2007
567,2004,0,1535,2,0,3,7,2010


In [32]:
test_data = np.array([2003,	854,	1710,	2,	1,	3,	8,	2008])
print(model.predict(test_data.reshape(1,8)))

[[182982.14]]


In [23]:
y_test.head()

,SalePrice
1053,144500
864,250580
185,475000
631,209500
567,214000
